# Future Mobility Days -- ADAC Challenge

Wie kann uns der technologische Fortschritt dabei helfen unsere alltäglichen Pendelwege angenehmer und effizienter zu gestalten? Welche Schlüsse lassen sich aus immer feiner aufgelösten Datensätzen ziehen?

In diesem Notebook erarbeiten wir eine Case Study, in der wir basierend auf Verkehrsdaten aus der Kleinstadt Landsberg am Lech illustrieren, wie eine intelligente Auto-Ampel Kommunikation zu einem erheblich verbessertem Verkehrsfluss führen kann.

Der typische Autofahrer fährt heutzutage mit voller Geschwindigkeit auf eine rote Ampel zu, bremst dann rapide ab und muss bei der Grünschaltung wieder aus dem Stand beschleunigen. Dies ist aus ökonomischer, ökologischer und verkehrstechnischer Sicht in höchstem Maße ineffizient. 

Wir schlagen vor, dass Ampelschaltungen den genauen Beginn der nächsten Grünphase an die Autos übermitteln. Diese können dann die Anfahrt zur Ampel so timen, dass keine Wartezeit an der Ampel auftritt und diese mit voller Geschwindigkeit überquert werden kann. Durch die zunehmende Bedeutung des autonomen Fahrens, kann die hier vorgestellte intelligente Auto-Ampel Kommunikation einfach und auf großen Skalen implementiert werden.

## Datenvorverarbeitung

Die zur Verfügung gestellten Daten betreffen Verkehrsbewegungen in ganz Bayern über den Zeitraum von einem Monat. Um den 32 GB großen Datensatz handhaben zu können, laden wir nur die für die Case Study relevanten Attribute.

In [ ]:
import pandas as pd

important_cols = ['assetId', 'latitude', 'longitude', 'town', 'velocity', 'timestamputc']
df = pd.read_csv('../data/raw/Bayern_2016_03.csv',  encoding = "ISO-8859-1", sep = ";", 
                usecols = important_cols)

Aus dem bayernweiten Datensatz wird nun das Teilgebiet zu Landsberg am Lech extrahiert und gesichert. Außerdem werden Längen- und Breitengrad in ein Floating-point Format umgrechnet.

In [96]:
lale = ['LANDSBERG' in w for w in df['town']]
landsberg = df[lale]

landsberg['lat'] = landsberg['latitude'] * 1e-6
landsberg['lon'] = landsberg['longitude'] * 1e-6
landsberg.to_csv('../data/raw/landsberg.csv', header = True, index = False)

Für weitere Auswertungen kann nun direkt der Landsberg-spezifische Datensatz geladen werden.

In [5]:
landsberg = pd.read_csv('../data/raw/landsberg.csv',  encoding = "ISO-8859-1")

## Konzentration auf Verkehr im Stadtzentrum

Der Großteil der Verkehrsdaten erfasst den Autobahnverkehr im Norden der Stadt und ist deshalb für die Ampel-Case Study nicht relevant. Deshalb extrahieren wir im nächsten Fahrten, die durch das Stadtzentrum verlaufen.

In [149]:
[LAT_MIN, LAT_MAX, LON_MIN, LON_MAX] = [48.045, 48.05, 10.865, 10.88]

landsberg_center = landsberg[(landsberg['lat'] >  LAT_MIN) & (landsberg['lat'] <  LAT_MAX)
                           & (landsberg['lon'] > LON_MIN) & (landsberg['lon'] < LON_MAX)]

center_rides = landsberg_center['assetId'].unique()

Das Paket `Folium` ermöglicht es uns Beispielfahrten schnell zu visualisieren.

In [159]:
import folium

EXAMPLE_ID = center_rides[157]
LANDSBERG_CENTER_LAT = 48.0460
LANDSBERG_CENTER_LON =  10.87200
ZOOM = 16

example_ride = landsberg[landsberg['assetId'] == EXAMPLE_ID][['timestamputc', 'lat', 'lon']]

map_osm = folium.Map(location=[LANDSBERG_CENTER_LAT, LANDSBERG_CENTER_LON], zoom_start = ZOOM)
for each in example_ride[['lat','lon']].values:  
    folium.Marker(each).add_to(map_osm)
map_osm

Für die Case Study betrachten wir nur die Zufahrt an eine Ampel und die danachfolgende Weiterfahrt.

In [115]:
#Ampelzufahrt und Weiterfahrt
example = example_ride.iloc[19:35,:].reset_index(drop = True)

## Berechnung der Ampelanfahrtsstrategie

Im Folgenden berechnen wir eine verbesserte Ampelanfahrtsstrategie, sodass im Moment des Grünschaltens der Ampel das Auto diese mit voller Geschwindigkeit passieren kann. Dabei wird die Zufahrt zur Ampel zunächst verlangsamt, und dann wieder beschleunigt. Dadurch wird erreicht, dass das Fahrzeug die Ampel erst in der Grünphase erreicht, dann aber bereits die volle Geschwindigkeit hat.

### Feature-Transformation

Um die Anfahrtsstrategie zu implementieren, stellen wir zunächst Zeit und Distanz zum Ausgangspunkt als skalare Größen dar.

In [117]:
import numpy as np
pd.options.mode.chained_assignment = None 


example['time'] = pd.DatetimeIndex(example['timestamputc']).astype(np.int64)

from geopy.distance import vincenty

def distance(ride):
    coords = ride[['lat', 'lon']]
    coords_pairs = pd.concat([coords, coords.shift()], ignore_index = True, axis = 1).iloc[1:,:]
    distances = coords_pairs.apply(lambda row: vincenty(row[:2].values, row[2:].values).meters, axis = 1)
    return np.append([0], distances.cumsum().values)

example['distance'] = distance(example)

Für eine lineare Interpolation des Fahrzeugs benötigen wir außerdem Koordinaten aufeinanderfolgender Wegpunkte.

In [125]:
example['shift_distance'] = example['distance'].shift(-1).fillna(method = 'ffill')
example['shift_lat'] = example['lat'].shift(-1).fillna(method = 'ffill')
example['shift_lon'] = example['lon'].shift(-1).fillna(method = 'ffill')

### Formel Weglänge Anfahrtsstrategien

Nun berechnen wir Anfahrtsprofile mit variiender Komplexitätsstufe. Zunächst betrachten wir Fortbewegung mit konstanter Geschwindigkeit.

In [71]:
def loc_const_vel(v):
    return lambda t: t * v

Als nächstes haben wir eine Fortbewegung mit linearer Geschwindigkeitsänderung.

In [86]:
def loc_lin_vel(tmin, tmax, vmin, vmax):
    return lambda t: loc_const_vel(0.5 * (vmin + np.interp(t, [tmin, tmax], [vmin, vmax])))(t - tmin)

Schließlich die Bewegung nach einem Geschwindigkeitsprofil, das zunächst linear abnimmt und dann linear zunimmt.

In [87]:
def loc_bilin_vel(tmin, tmax, vmin, vmax):
    tmid = 0.5 * (tmin + tmax)
    vmid = 0.5 * (vmin + vmax)
    offset = loc_const_vel(vmid)(tmid-tmin)
    
    return lambda t: (loc_lin_vel(tmin, tmid, vmax, vmin)(t) if t < tmid 
                      else offset + loc_lin_vel(tmid, tmax, vmax, vmin)(t))

def final_loc(tmin, tmax, vmin, vmax):
    offset = loc_bilin_vel(tmin, tmax, vmin, vmax)(tmax) 
    return lambda t: (loc_bilin_vel(tmin, tmax, vmin, vmax)(t) if t < tmax 
                      else offset + loc_const_vel(vmax)(t - tmax))

### Berechnung Anfahrtsstrategie auf Realdaten

Um die Anfahrtsstrategie auf den Landsbergdaten zu implementieren, setzen wir eine Maximalgeschwindigkeit von 40 km/h an.

In [118]:
example_small = example.iloc[:10,:]
ampel_idx = 9
dx = example.loc[ampel_idx, 'distance']
tmin = example.loc[0, 'time'] 
tmax =  example.loc[ampel_idx, 'time']
dt = tmax - tmin

def comp_vmin(vmax, travel_dist, travel_time):
    return 2*travel_dist/travel_time - vmax

vmax = 40 / (3.6e9)
vmin = comp_vmin(vmax , dx, dt)

3.7028703799875299e-09

Nun ergänzen wir die Beispieldaten um die neue Anfahrtsstrategie.

In [126]:
example['new_dist'] = example['time'].apply(lambda x: final_loc(tmin, tmax, vmin,vmax)(x))
example['shift_new_dist'] = example['new_dist'].shift(-1).fillna(method = 'ffill')

#Nur Ampelzufahrt
example_small = example.iloc[:(ampel_idx + 1), :]

### Rücktransformation auf Koordinatenebene

Die neue Anfahrtsstrategie wird nun auf die Geo-Koordinaten zurückgerechnet

In [143]:
def compute_location(dist, ride):
    mask = ride.apply(lambda row: (dist >= row['distance']) & (dist <= row['shift_distance']), axis = 1)
    loc_row = ride[mask].iloc[0]    
    return [np.interp(dist, [loc_row['distance'], loc_row['shift_distance']], 
                      [loc_row['{}'.format(lalo)], loc_row['shift_{}'.format(lalo)]]) for lalo in ['lat', 'lon']]


In [145]:
#Vermeide Probleme an Definitionsrändern
MIN_VAL = .1
MAX_VAL = example.iloc[-1, :]['distance'] - .1

anticip_example = example['new_dist'].apply(lambda x: compute_location(min(max(MIN_VAL, x), MAX_VAL), example)).values

### Visualisierung der verbesserten Anfahrtsstrategie

Durch die verbesserte Anfahrtsstrategie (grün) wird im Vergleich zum Original (rot) in der gleichen Zeit ein deutlich weiterer Weg zurückgelegt.

In [161]:
from folium.plugins import MarkerCluster
from folium.features import DivIcon
map_osm = folium.Map(location=[LANDSBERG_CENTER_LAT, LANDSBERG_CENTER_LON], zoom_start = ZOOM)

for each in example[['lat','lon']].values[:-2]:  
    folium.Marker(each , icon = folium.Icon(color = 'red')).add_to(map_osm)
    
for a,each in enumerate(anticip_example):  
    folium.Marker(each , icon = folium.Icon(color = 'green')).add_to(map_osm)
map_osm